In [4]:
import pandas as pd
import numpy as np

In [5]:
# !pip install pandas

## Loading the groundtruth data

In [6]:
data = pd.read_csv(r"E:\Work\Data_Science\Projects\Custom_NER\data\ner.csv", encoding='cp1252', on_bad_lines='warn')
data.head()

Skipping line 281837: expected 25 fields, saw 34



,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [7]:
data.sentence_idx.value_counts()

2549.0     140
11994.0    132
608.0      124
5805.0     122
6344.0     120
          ... 
37093.0      2
8412.0       2
39874.0      2
40249.0      2
38917.0      1
Name: sentence_idx, Length: 35177, dtype: int64

In [8]:
sentences = data.sentence_idx.value_counts().index[:10000]
#sentences = sentences.index
#sentences

In [9]:
len(sentences)

10000

In [10]:
new_data = data[data['sentence_idx'].isin(sentences)]
new_data.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [11]:
new_data.tag.value_counts()

O        424516
B-geo     17934
B-org      9841
B-tim      9453
I-per      8744
I-org      8570
B-per      8208
B-gpe      8075
I-geo      3876
I-tim      3057
B-art       255
B-eve       207
I-eve       168
I-art       156
I-gpe       136
B-nat       124
I-nat        46
Name: tag, dtype: int64

In [10]:
# # parsing data 
# data_json = []
# for i in sentences:
#     s = ""
#     entities = []
#     sub_data = new_data[new_data['sentence_idx']==i]
#     beg = 0
#     for j in sub_data.index:
#         word = sub_data.loc[j,'word']
#         s = s+' '+word
#         tag = sub_data.loc[j,'tag']
#         if tag != 'O':
#             start = beg+s[beg:].find(word)
#             end = start+len(word)
#             beg=end
#             entities.append([start, end, tag])
#     data_json.append([s, {'entities':entities}])
# #print(data_json)

    

### Parsing the data to follow this convention:-
 ["this is our text", {'entities':[[start, end, label], [start, end, label]]},</br>
"this is our text", {'entities':[[start, end, label]]}...]

In [11]:
# parsing data 
data_json = []
for i in sentences:
    s = ""
    beg = 0
    entities = []
    sub_data = new_data[new_data['sentence_idx']==i]
    for j in sub_data.index:
        word = sub_data.loc[j,'word']
        s = s+word+' '
        start = beg
        end = start+len(word)
        beg = end+1
        tag = sub_data.loc[j,'tag']
        if tag != 'O':
            entities.append([start, end, tag])
    data_json.append([s, {'entities':entities}])
#print(data_json)

    

In [12]:
' Last week , Mr. Annan said the U.N. force will have to be bigger and better equipped , which , he said , will require the participation of big and powerful countries with highly trained troops - including the U.S. U.S. officials have said they hope to arrange a U.N. mission to Darfur by the end of this month . Last week , Mr. Annan said the U.N. force will have to be bigger and better equipped , which , he said , will require the participation of big and powerful countries with highly trained troops - including the U.S. U.S. officials have said they hope to arrange a U.N. mission to Darfur by the end of this month .'[17:22]

'Annan'

In [13]:
len(data_json)

10000

In [14]:
#data_json

In [15]:
# train dev test split
import random
test_list = random.sample(range(0, 10000), 1500)
test_data_json = [ele for i,ele in enumerate(data_json) if i in test_list]
train_data_json = [ele for i,ele in enumerate(data_json) if i not in test_list]
dev_list = random.sample(range(0, 8500), 1500)
dev_data_json = [ele for i,ele in enumerate(train_data_json) if i in dev_list]
train_data_json = [ele for i,ele in enumerate(train_data_json) if i not in dev_list]
len(test_data_json), len(dev_data_json), len(train_data_json)

(1500, 1500, 7000)

In [16]:
import json
test_data_json = json.dumps(test_data_json)
dev_data_json = json.dumps(dev_data_json)
train_data_json = json.dumps(train_data_json)
data_json = json.dumps(data_json)

In [18]:
# save in json format
with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\all\data_json.json', 'w') as f:
    f.write(data_json)

with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\train\train_data_json.json', 'w') as f:
    f.write(train_data_json)

with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\dev\dev_data_json.json', 'w') as f:
    f.write(dev_data_json)
    
with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\test\test_data_json.json', 'w') as f:
    f.write(test_data_json)

In [2]:
# read the json back
import json
with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\train\train_data_json.json', 'r') as f:
    train_data_json = json.load(f)

with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\dev\dev_data_json.json', 'r') as f:
    dev_data_json = json.load(f)
    
with open(r'E:\Work\Data_Science\Projects\Custom_NER\data\Train\test\test_data_json.json', 'r') as f:
    test_data_json = json.load(f)
    

## Function to convert json to spacy format for spacy NER 

In [22]:
# convert to spacy format of train and dev, save them
import spacy
from spacy.tokens import DocBin
def json2ner_spacy(ip, dest):
    nlp = spacy.blank('en')
    db = DocBin()
    c = 0
    for text, annotation in ip:
        c+=1
        doc = nlp(text)
        ents = []
        for start, end, label in annotation['entities']:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(dest)
    
json2ner_spacy(train_data_json, r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_ner\train\train.spacy")
json2ner_spacy(dev_data_json, r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_ner\dev\dev.spacy")

## Function to convert spacy ner format to spacy Span Categorizer format

In [4]:
from pathlib import Path
import typer
import spacy
from spacy.tokens import DocBin


def ner_spacy2spancat_spacy(src, dest):
    """
    Set the NER data into the doc.spans, under a given key.
    The SpanCategorizer component uses the doc.spans, so that it can work with
    overlapping or nested annotations, which can't be represented on the
    per-token level.
    """
    nlp = spacy.blank('en')
    span_key = 'sc'
    docbin = DocBin().from_disk(src)
    docs = list(docbin.get_docs(nlp.vocab))
    for doc in docs:
        doc.spans[span_key] = list(doc.ents)
    DocBin(docs=docs).to_disk(dest)

ner_spacy2spancat_spacy(r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_ner\train\train.spacy", 
                   r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_spancat\train\train.spacy")
ner_spacy2spancat_spacy(r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_ner\dev\dev.spacy", 
                   r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_spancat\dev\dev.spacy")

In [3]:
# # this is a alternate solution to the previous function, here we directly convert json data to spancat spacy data

# def json2spancat_spacy(ip, dest):
#     """
#     our json follows ner annotations, we have to 
#     """
#     span_key = 'sc'
#     nlp = spacy.blank('en')
#     db = DocBin()
#     c = 0
#     for text, annotation in ip:
#         c+=1
#         doc = nlp(text)
#         spans = []
#         for start, end, label in annotation['entities']:
#             span = doc.char_span(start, end, label=label)
#             spans.append(span)
#         doc.spans[span_key] = spans
#         db.add(doc)
#     db.to_disk(dest)
    
# # json2ner_spacy(train_data_json, r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_spancat\train\train.spacy")
# # json2ner_spacy(dev_data_json, r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_spancat\dev\dev.spacy")

# Create classifier data

In [20]:
train_data_json[0]

['The Hunter agreed , but said : " If you desire to conquer the Stag , you must permit me to place this piece of iron between your jaws , so that I may guide you with these reins , and allow this saddle to be placed upon your back so that I may keep steady upon you as we follow after the enemy . " The Hunter agreed , but said : " If you desire to conquer the Stag , you must permit me to place this piece of iron between your jaws , so that I may guide you with these reins , and allow this saddle to be placed upon your back so that I may keep steady upon you as we follow after the enemy . " ',
 {'entities': []}]

In [26]:
labels = new_data.tag.unique()
labels

array(['O', 'B-geo', 'B-gpe', 'B-per', 'B-org', 'I-org', 'B-tim', 'B-art',
       'I-art', 'I-per', 'I-geo', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [35]:
cats = {}
for k in labels:
    cats[k]=0
print(cats)

{'O': 0, 'B-geo': 0, 'B-gpe': 0, 'B-per': 0, 'B-org': 0, 'I-org': 0, 'B-tim': 0, 'B-art': 0, 'I-art': 0, 'I-per': 0, 'I-geo': 0, 'I-gpe': 0, 'I-tim': 0, 'B-nat': 0, 'B-eve': 0, 'I-eve': 0, 'I-nat': 0}


In [42]:
# training data
training_textcat_json = []
for text, ents in train_data_json:
    copy_cat = cats.copy()
    available_cats = [i[-1] for i in ents['entities']]
    for l in available_cats:
        copy_cat[l]=1
    data = [text, {'cats':copy_cat}]
    training_textcat_json.append(data)

print(training_textcat_json[:2])

[['The Hunter agreed , but said : " If you desire to conquer the Stag , you must permit me to place this piece of iron between your jaws , so that I may guide you with these reins , and allow this saddle to be placed upon your back so that I may keep steady upon you as we follow after the enemy . " The Hunter agreed , but said : " If you desire to conquer the Stag , you must permit me to place this piece of iron between your jaws , so that I may guide you with these reins , and allow this saddle to be placed upon your back so that I may keep steady upon you as we follow after the enemy . " ', {'cats': {'O': 0, 'B-geo': 0, 'B-gpe': 0, 'B-per': 0, 'B-org': 0, 'I-org': 0, 'B-tim': 0, 'B-art': 0, 'I-art': 0, 'I-per': 0, 'I-geo': 0, 'I-gpe': 0, 'I-tim': 0, 'B-nat': 0, 'B-eve': 0, 'I-eve': 0, 'I-nat': 0}}], ['One of them , turning about , said to him : " That is the very reason why we are so cautious ; for if you yesterday treated us better than the Goats you have had so long , it is plain a

In [56]:
print(training_textcat_json[2:3])

[["The two most traumatic experiences in the nation 's history were the Civil War ( 1861 - 65 ) , in which a northern Union of states defeated a secessionist Confederacy of 11 southern slave states , and the Great Depression of the 1930s , an economic downturn during which about a quarter of the labor force lost its jobs . The two most traumatic experiences in the nation 's history were the Civil War ( 1861 - 65 ) , in which a northern Union of states defeated a secessionist Confederacy of 11 southern slave states , and the Great Depression of the 1930s , an economic downturn during which about a quarter of the labor force lost its jobs . ", {'cats': {'O': 0, 'B-geo': 1, 'B-gpe': 0, 'B-per': 0, 'B-org': 1, 'I-org': 1, 'B-tim': 1, 'B-art': 0, 'I-art': 0, 'I-per': 0, 'I-geo': 0, 'I-gpe': 0, 'I-tim': 0, 'B-nat': 0, 'B-eve': 0, 'I-eve': 0, 'I-nat': 0}}]]


In [40]:
len(training_textcat_json)

7000

In [78]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank('en')
db = DocBin()

In [79]:
labels

array(['O', 'B-geo', 'B-gpe', 'B-per', 'B-org', 'I-org', 'B-tim', 'B-art',
       'I-art', 'I-per', 'I-geo', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [80]:
for text, annots in training_textcat_json[:100]:
    doc = nlp(text)
    doc.cats = annots['cats']
    db.add(doc)
         

In [82]:
doc.cats

{'O': 0,
 'B-geo': 1,
 'B-gpe': 1,
 'B-per': 1,
 'B-org': 1,
 'I-org': 1,
 'B-tim': 0,
 'B-art': 0,
 'I-art': 0,
 'I-per': 1,
 'I-geo': 0,
 'I-gpe': 0,
 'I-tim': 0,
 'B-nat': 0,
 'B-eve': 0,
 'I-eve': 0,
 'I-nat': 0}

In [81]:
db.to_disk(r"E:\Work\Data_Science\Projects\Custom_NER\data\Train_textcat\train\train.spacy")

In [ ]:
# convert to spacy format of train and dev, save them
import spacy
from spacy.tokens import DocBin
def json2textcat_spacy(ip, dest):
    nlp = spacy.blank('en')
    db = DocBin()
    c = 0
    for text, annotation in ip:
        c+=1
        doc = nlp(text)
        ents = []
        for start, end, label in annotation['entities']:
            span = doc.char_span(start, end, label=label)
            ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(dest)